In [ ]:
# Note Parsing
import re

# Audio
from IPython.lib.display import Audio
import soundfile as sf

# Array Processing
import numpy as np


####**Note Duration**   
whole                         - 16   
half                          - 8   
dotted quarter                - 6   
quarter                       - 4    
dotted one-eight              - 3   
one-eight                     - 2   
one-sixteenth                 - 1   

https://musescore.com/user/8784906/scores/4127521

In [ ]:
# 2d list with key and duration
melody = [["d5",6],["c5#",2],["b4",3],["a4",1],["f4#",3],["d5",1],
          ["c5#",6],["b4",2],["a4",3],["f4#",1],["d4",3],["c5#",1],
          ["b4",6],["a4",2],["g4",3],["e4",1],["c4#",3],["b4",1],
          ["a4",8],["g4",8]]

chords = [["e3",4],["e4",4],["e3",4],["e4",4],
          ["d3",4],["d4",4],["d3",4],["d4",4],
          ["e3",4],["e4",4],["e3",4],["e4",4],
          ["d3",4],["e4",4],["a3",8]]

base = [["",4],["e2",4],["e2",4],["e2",4],
       ["",4],["d2",4],["d2",4],["d2",4],
       ["",4],["e2",4],["e2",4],["e2",4],
       ["d2",8],["e2",8]]

In [ ]:
def note_num(note):
  n = 0
  if re.search("^([a-g])([1-5])([#|b|])?",note):
    key = re.search("[a-g]",note).group()
    oct = int(re.search("[1-5]",note).group())

    if oct == 1:
        n = 4
    elif oct == 2:
        n = 16
    elif oct == 3:
        n = 28
    elif oct == 4:
        n = 40
    elif oct == 5:
        n = 52
    else: n = 0

    if key == "c":
        n += 0
    elif key == "d":
        n += 2
    elif key == "e":
        n += 4
    elif key == "f":
        n += 5
    elif key == "g":
        n += 7
    elif key == "a":
        n += 9
    elif key == "b":
        n += 11
    else: n = 0

    if re.search("^([a-g])([1-5])([#])",note):
      n += 1
    elif re.search("^([a-g])([1-5])([b])",note):
      n -= 1
  else:
    pass

  return n


In [ ]:
def get_freq(signal):
    if signal >=61:
      f = 880 * (2** ((signal-61)/12))
      # print('number = %d ,frequency = %f'  % (signal , f ))
      return f, 1

    elif signal >= 49 and signal < 61:
      f = 440 * (2** ((signal-49)/12))
      # print('number = %d ,frequency = %f'  % (signal , f ))
      return f, 1

    elif signal >= 37 and signal < 49:
      f = 220 * (2** ((signal-37)/12))
      # print('number = %d ,frequency = %f'  % (signal , f ))
      return f, 1

    elif signal >= 25 and signal < 37:
      f = 110 * (2** ((signal-25)/12))
      # print('number = %d ,frequency = %f'  % (signal , f ))
      return f, 8

    elif signal >= 13 and signal < 25:
      f = 55 * (2** ((signal-13)/12))
      # print('number = %d ,frequency = %f'  % (signal , f ))
      return f, 16
    else: return 0, 0

In [ ]:
asignals = []
bsignals = []
csignals = []


def get_beat(lis):
  base = 4
  for i in range(len(lis)):
    if lis[i][1] < base:
        base = lis[i][1]
    return int(lis[i][1] / base)

def note_parser(signal, notes, beat):
  for i in range(len(notes)):
    num = note_num(notes[i][0])
    signal.extend([num for i in range(beat*notes[i][1])])

In [ ]:
beat = get_beat(melody)
note_parser(asignals, melody, beat)
note_parser(bsignals, chords, beat)
note_parser(csignals, base, beat)

In [ ]:
music = [asignals, bsignals, csignals]
most= music[np.argmax([len(l) for l in music])]

bsignals.extend([0 for i in range(len(most)-len(bsignals))])
csignals.extend([0 for i in range(len(most)-len(csignals))])

print(asignals)
print(bsignals)
print(csignals)

[54, 54, 54, 54, 54, 54, 53, 53, 51, 51, 51, 49, 46, 46, 46, 54, 53, 53, 53, 53, 53, 53, 51, 51, 49, 49, 49, 46, 42, 42, 42, 53, 51, 51, 51, 51, 51, 51, 49, 49, 47, 47, 47, 44, 41, 41, 41, 51, 49, 49, 49, 49, 49, 49, 49, 49, 47, 47, 47, 47, 47, 47, 47, 47]
[32, 32, 32, 32, 44, 44, 44, 44, 32, 32, 32, 32, 44, 44, 44, 44, 30, 30, 30, 30, 42, 42, 42, 42, 30, 30, 30, 30, 42, 42, 42, 42, 32, 32, 32, 32, 44, 44, 44, 44, 32, 32, 32, 32, 44, 44, 44, 44, 30, 30, 30, 30, 44, 44, 44, 44, 37, 37, 37, 37, 37, 37, 37, 37]
[0, 0, 0, 0, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 0, 0, 0, 0, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 0, 0, 0, 0, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 18, 18, 18, 18, 18, 18, 18, 18, 20, 20, 20, 20, 20, 20, 20, 20]


In [ ]:
fs = 8000          # frequency sampling from est. freq. of human hearing

t = np.arange(0,0.25,1/fs)      # length of beat; larger the slower

signals1 = asignals
signals2 = bsignals
signals3 = csignals

output1 = output2 = output3 = np.cos(2*np.pi*0*t)   # starting blank signal

for i in range(len(signals1)):    # Melody
  f, v = get_freq(signals1[i])
  x= v * np.cos(2*np.pi*f*t)
  # print(f,x)
  output1 = np.concatenate([output1,x])

for i in range(len(signals2)):    # Harmony
  f, v = get_freq(signals2[i])
  x= v * np.cos(2*np.pi*f*t)

  output2 = np.concatenate([output2,x])

for i in range(len(signals3)):    # base
  f, v = get_freq(signals3[i])
  x= v * np.cos(2*np.pi*f*t)

  output3 = np.concatenate([output3,x])

final = output1 + output1 + output2 + output3

# sf.write('sinemusic.wav' ,final,fs)
Audio(final,rate = fs, autoplay = True)
# Audio(output1,rate = fs, autoplay = True)
# Audio(output2,rate = fs, autoplay = True)
# Audio(output3,rate = fs, autoplay = True)
